In [1]:
from __future__ import print_function
from pythtb import *
import numpy as np
import matplotlib.pyplot as plt
from itertools import product as itp

In [2]:
# define the lattice vectors and the coordinates of orbitals
lat = [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]
orb = [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]

In [3]:
# define your model by making a 2D tight-binding checkboard model
my_model = tb_model(3,3,lat, orb)

In [4]:
# set model parameters
delta = 0.0
t = -0.6

In [5]:
# set on-site energies

# the first parameter includes a list of on-site energies for each orbital
# if this function is never called, the on-site energy is assumed to be zero
my_model.set_onsite([delta, delta, delta])

In [6]:
# set hoppings (one for each connected pair of orbitals)
# (hopping amplitude, i (index of orbital in the home unit cell), j, [lattice vector to cell containing j])

# hopping from the "first" orbital in the home unit cell to the "zeroth" orbital which is still in the home unit
# cell (indicated by the lattice vector [0, 0])
my_model.set_hop(t, 0, 0, [0, 1, 0])
my_model.set_hop(t, 0, 0, [1, 0, 0])

my_model.set_hop(t, 1, 1, [0, 1, 0])
my_model.set_hop(t, 1, 1, [0, 0, 1])

my_model.set_hop(t, 2, 2, [1, 0, 0])
my_model.set_hop(t, 2, 2, [0, 0, 1])

In [7]:
# print or display the tight-binding model
my_model.display()

---------------------------------------
report of tight-binding model
---------------------------------------
k-space dimension           = 3
r-space dimension           = 3
number of spin components   = 1
periodic directions         = [0, 1, 2]
number of orbitals          = 3
number of electronic states = 3
lattice vectors:
 #  0  ===>  [     1.0 ,     0.0 ,     0.0 ]
 #  1  ===>  [     0.0 ,     1.0 ,     0.0 ]
 #  2  ===>  [     0.0 ,     0.0 ,     1.0 ]
positions of orbitals:
 #  0  ===>  [     0.0 ,     0.0 ,     0.0 ]
 #  1  ===>  [     0.0 ,     0.0 ,     0.0 ]
 #  2  ===>  [     0.0 ,     0.0 ,     0.0 ]
site energies:
 #  0  ===>       0.0
 #  1  ===>       0.0
 #  2  ===>       0.0
hoppings:
<  0 | H |  0 + [  0 ,  1 ,  0 ] >     ===>     -0.6 +     0.0 i
<  0 | H |  0 + [  1 ,  0 ,  0 ] >     ===>     -0.6 +     0.0 i
<  1 | H |  1 + [  0 ,  1 ,  0 ] >     ===>     -0.6 +     0.0 i
<  1 | H |  1 + [  0 ,  0 ,  1 ] >     ===>     -0.6 +     0.0 i
<  2 | H |  2 + [  1 ,  0 ,  

In [8]:
from triqs.lattice.tight_binding import TBLattice
import sympy as sp

def sympyfy(tb_lat_obj, numerical = True):
    r"""
    returns the analytical form of the momentum space hamiltonian of the tight-binding model 
    from a tight-binding lattice object, by utilizing Fourier series
    
    Parameters
    ----------
    tb_lat_obj: triqs TBLattice object
        triqs tight binding object
    numerical: boolean, default = True
        a boolean which will cause the function will return a numerical Hamiltonian, when true, and 
        an analytical Hamiltonian otherwise
    
    Returns
    -------
    Hk_numerical: NumPy Array
        the hamiltonian of the tight-binding model in momentum space in numerical form
    Hk_analytical: NumPy Array
        the hamiltonian of the tight-binding model in momentum space in reduced analytical or symbolic form
    """

    I = sp.I

    # axis directions in momentum space (dimension in momentum space is 3)
    kx, ky, kz = sp.symbols("kx ky kz")
    
    # individual components of the lattice constants
    L1_1, L1_2, L1_3, L2_1, L2_2, L2_3, L3_1, L3_2, L3_3 = sp.symbols("L1_1 L1_2 L1_3 L2_1 L2_2 L2_3 L3_1 L3_2 L3_3")
    
    # making a matrix out of the momentum space axial directions
    k_space_matrix = sp.Matrix([kx, ky, kz])
    
    # lattice constants
    lattice = sp.Matrix([[L1_1, L1_2, L1_3],
                         [L2_1, L2_2, L2_3],
                         [L3_1, L3_2, L3_3]])
    
    # the number of orbitals
    num_orb = tb_lat_obj.n_orbitals
    
    # the inter-orbital electron hoppings
    TB_lat_obj_hops = tb_lat_obj.hoppings 
    
    # maximum hopping distances of electrons in each of the axial directions
    max_x, max_y, max_z = list(np.max(np.array(list(TB_lat_obj_hops.keys())), axis = 0))
    
    # number of cells involved in the hopping of electrons in each of the axial directions
    num_cells_x, num_cells_y, num_cells_z = [2 * max_coord + 1 for max_coord in [max_x, max_y, max_z]]
    
    # basis of the 5D tensor real-space Hamiltonian
    Hrij = np.zeros((num_cells_x, num_cells_y, num_cells_z, num_orb, num_orb), dtype = sp.exp)
    
    # looping through the hopping parameters of the electrons involved in the inter-orbital hoppings
    # key represents the cell coordinates of where the electrons hop to relative to the home unit cell
    # hopping represents the matrix with the embedded hopping amplitudes
    for key, hopping in TB_lat_obj_hops.items():
        rx, ry, rz = key
        Hrij[rx + max_x, ry + max_y, rz + max_z] = hopping
    
    # basis of the exponential term in the calculation of Hk
    Hexp = np.empty_like(Hrij, dtype = sp.exp)
    
    # actual fourier transform
    for xi, yi, zi in itp(range(num_cells_x), range(num_cells_y), range(num_cells_z)):
        r = np.array([xi - max_x, yi - max_y, zi - max_z])
        r = lattice.dot(r)
        eikr = sp.exp(-I * k_space_matrix.dot(r))
        Hexp[xi, yi, zi, :, :] = eikr
     
    # summation over all real space axes
    Hk = np.sum(Hrij * Hexp, axis = (0, 1, 2))
    
    # simplifying and rewriting Hk in terms of cosines
    for i, j in itp(range(num_orb), repeat = 2):
        Hk[i, j] = Hk[i, j].rewrite(sp.cos).simplify()
    
    # the expanded analytical form of Hk
    Hk_analytical = sp.Matrix(Hk)
    
    # symbols for the dot product between the lattice constants and the momentum space matrix
    ak, bk, ck = sp.symbols("ak bk ck")
    
    # reducing our analytical Hk form using the lattice constants-k-space matrix dot product
    Hk_analytical = Hk_analytical.subs(L1_1*kx + L2_1*ky + L3_1*kz, ak)
    Hk_analytical = Hk_analytical.subs(L1_2*kx + L2_2*ky + L3_2*kz, bk)
    Hk_analytical = Hk_analytical.subs(L1_3*kx + L2_3*ky + L3_3*kz, ck)
    
    # our reduced analytical expression as a NumPy array
    Hk_analytical = np.array(Hk_analytical)
    
    # foundation of numerical form of Hk
    Hk_numerical = sp.Matrix(Hk)

    TB_lat_obj_units = tb_lat_obj.units
    TB_lat_obj_units_transpose = np.transpose(TB_lat_obj_units)

    # lattice constants
    a = TB_lat_obj_units_transpose[0]
    b = TB_lat_obj_units_transpose[1]
    c = TB_lat_obj_units_transpose[2]

    # numerical dot product between lattice constants and momentum space matrix
    ak_numerical = a.dot(k_space_matrix)[0]
    bk_numerical = b.dot(k_space_matrix)[0]
    ck_numerical = c.dot(k_space_matrix)[0]
    
    # performing numerical subsitutions
    Hk_numerical = Hk_numerical.subs(L1_1*kx + L2_1*ky + L3_1*kz, ak_numerical)
    Hk_numerical = Hk_numerical.subs(L1_2*kx + L2_2*ky + L3_2*kz, bk_numerical)
    Hk_numerical = Hk_numerical.subs(L1_3*kx + L2_3*ky + L3_3*kz, ck_numerical)
    
    # our reduced numerical expression as a NumPy array
    Hk_numerical = np.array(Hk_numerical)

    # check to see whether user wants analytical expression or numerical expression
    if numerical:
        return Hk_numerical
    return Hk_analytical

Starting serial run at: 2023-07-07 10:29:27.284326


In [9]:
from triqs.lattice.utils import TB_from_pythTB
tb_triqs = TB_from_pythTB(my_model)

# for key, hopping in tb_triqs.hoppings.items():
#     print(key)
#     print(hopping)
print(sympyfy(tb_triqs))
print(sympyfy(tb_triqs, False))


/Users/munenekariuki/opt/anaconda3/envs/TRIQS/lib/python3.11/site-packages/sympy/matrices/matrices.py:1401: SymPyDeprecationWarning: 

Using the dot method to multiply non-row/column vectors is
deprecated. Use * or @ to perform matrix multiplication.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-matrix-dot-non-vector
for details.

This has been deprecated since SymPy version 1.2. It
will be removed in a future version of SymPy.

  return self.dot(Matrix(b))


[[-1.2*cos(1.0*kx) - 1.2*cos(1.0*ky) 0 0]
 [0 -1.2*cos(1.0*ky) - 1.2*cos(1.0*kz) 0]
 [0 0 -1.2*cos(1.0*kx) - 1.2*cos(1.0*kz)]]
[[-1.2*cos(ak) - 1.2*cos(bk) 0 0]
 [0 -1.2*cos(bk) - 1.2*cos(ck) 0]
 [0 0 -1.2*cos(ak) - 1.2*cos(ck)]]


In [10]:
print(tb_triqs.units)
print(type(tb_triqs))

kx, ky, kz = sp.symbols("kx ky kz")
kSpaceMatrix = sp.Matrix([kx, ky, kz])

transposeUnits = np.transpose(tb_triqs.units)
A = transposeUnits[0]
AK = A.dot(kSpaceMatrix)
print(AK[0])

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
<class 'triqs.lattice.tight_binding.TBLattice'>
1.0*kx


In [14]:
# # axis directions in momentum space (dimension in momentum space is 3)
# kx, ky, kz = sp.symbols("kx ky kz")

# # lattice matrix builder
# L1_1, L1_2, L1_3, L2_1, L2_2, L2_3, L3_1, L3_2, L3_3 = sp.symbols("L1_1 L1_2 L1_3 L2_1 L2_2 L2_3 L3_1 L3_2 L3_3")

# # converting the momentum space axial directions and elements of the lattice space matrix into matrices
# kSpaceMatrix = sp.Matrix([kx, ky, kz])

# lattice = sp.Matrix([[L1_1, L1_2, L1_3],
#                      [L2_1, L2_2, L2_3],
#                      [L3_1, L3_2, L3_3]])

# Hk = sympyfy(tb_triqs)
# ak, bk, ck = sp.symbols("ak bk ck")
# # expr.subs(x, y)
# Hk = Hk.subs(L1_1*kx + L2_1*ky + L3_1*kz, ak)
# Hk = Hk.subs(L1_2*kx + L2_2*ky + L3_2*kz, bk)
# Hk = Hk.subs(L1_3*kx + L2_3*ky + L3_3*kz, ck)
# print(Hk)


In [15]:
# generate the k-point path and labels
path= [[0.0,0.0,0.0],[0.5,0.0, 0.0],[0.5,0.5,0.0],[0.0,0.0,0.0]]
label = (r'$\Gamma $',r'$X$', r'$M$', r'$\Gamma $')
(k_vec, k_dist, k_node) = my_model.k_path(path, 101, report=False)


In [16]:
# import triqs.lattice.utils
# from triqs.lattice.tight_binding import TBLattice
from triqs.lattice.utils import TB_from_pythTB
tb_triqs = TB_from_pythTB(my_model)
# print(tb_triqs)


for key in tb_triqs.hoppings.keys():
    print(key)
    print(tb_triqs.hoppings[key].real)
    print()

(0, 0, 0)
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

(0, 1, 0)
[[-0.6  0.   0. ]
 [ 0.  -0.6  0. ]
 [ 0.   0.   0. ]]

(0, -1, 0)
[[-0.6  0.   0. ]
 [ 0.  -0.6  0. ]
 [ 0.   0.   0. ]]

(1, 0, 0)
[[-0.6  0.   0. ]
 [ 0.   0.   0. ]
 [ 0.   0.  -0.6]]

(-1, 0, 0)
[[-0.6  0.   0. ]
 [ 0.   0.   0. ]
 [ 0.   0.  -0.6]]

(0, 0, 1)
[[ 0.   0.   0. ]
 [ 0.  -0.6  0. ]
 [ 0.   0.  -0.6]]

(0, 0, -1)
[[ 0.   0.   0. ]
 [ 0.  -0.6  0. ]
 [ 0.   0.  -0.6]]



In [17]:
# solve for the eigenenergies of the Hamiltonian on the set of k-points above
evals = my_model.solve_all(k_vec)
print(evals)

[[-2.4        -2.4        -2.4        -2.4        -2.4        -2.4
  -2.4        -2.4        -2.4        -2.4        -2.4        -2.4
  -2.4        -2.4        -2.4        -2.4        -2.4        -2.4
  -2.4        -2.4        -2.4        -2.4        -2.4        -2.4
  -2.4        -2.4        -2.4        -2.4        -2.4        -2.4
  -2.39342627 -2.37377712 -2.34126782 -2.29625455 -2.23923048 -2.17082039
  -2.09177379 -2.00295673 -1.9053423  -1.8        -1.68808397 -1.57082039
  -1.44949403 -1.32543416 -1.2        -1.07456584 -0.95050597 -0.82917961
  -0.71191603 -0.6        -0.4946577  -0.39704327 -0.30822621 -0.22917961
  -0.16076952 -0.10374545 -0.05873218 -0.02622288 -0.00657373  0.
  -0.00352104 -0.01406349 -0.03156549 -0.05592433 -0.08699706 -0.12460133
  -0.16851647 -0.21848477 -0.27421298 -0.33537409 -0.40160916 -0.47252951
  -0.54771894 -0.62673622 -0.70911764 -0.79437975 -0.8820222  -0.97153067
  -1.06237989 -1.15403672 -1.24596328 -1.33762011 -1.42846933 -1.5179778
  -1.605

In [22]:
# # plotting of band structure
# print('Plotting the band structure...')
# fig, ax = plt.subplots()
# ax.set_xlim(k_node[0], k_node[-1])
# ax.set_xticks(k_node)
# ax.set_xticklabels(label)
# print(k_node)

# for n in range(len(k_node)):
#     ax.axvline(x=k_node[n], linewidth=0.5, color='k')

# # plot the bands
# for n in range(2):
#     ax.plot(k_dist, evals[n])

# # put title
# ax.set_title('Checkerboard band structure')
# ax.set_xlabel('Path in k-space')
# ax.set_ylabel('Band energy')

# fig.savefig('2D Tight-Binding Checkerboard Model.pdf')